In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType
from delta.tables import DeltaTable

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "custos"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_custo_chamado"
TABELA_INVALIDOS_DESTINO = "tb_custo_chamado_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"
nome_tabela_invalidos = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}"

df_bronze = spark.read.table(nome_tabela_origem)

max_dt_ingestao = (
    df_bronze
    .agg(F.max(F.col("ingestion_timestamp")).alias("max_ts"))
    .first()["max_ts"]
)

df_filtrado = df_bronze.filter(
    F.col("ingestion_timestamp") == F.lit(max_dt_ingestao)
)

In [0]:
df_limpo = (
    df_filtrado
    .select(
        F.col("id_custo").alias("cd_custo"),
        F.col("id_chamado").alias("cd_chamado"),
        F.translate(
            F.regexp_replace(
                F.col("custo"),
                r"R\$|reais|real", 
                ""
            ),
            ",",
            "."
        ).cast(DecimalType(12, 10)).alias("vl_custo"),
        F.current_timestamp().alias("dt_ingestion"),
        F.col("origem").alias("dc_origem")
    )
    .dropDuplicates(["cd_custo"])
)

df_validacao = (
    df_limpo
    .withColumn("flag_id_valido", F.col("cd_custo").isNotNull())
    .withColumn("flag_chamado_valido", F.col("cd_chamado").isNotNull())
    .withColumn("flag_valor_valido", 
        F.col("vl_custo").isNotNull() & (F.col("vl_custo") > 0)
    )
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_id_valido") &
            F.col("flag_chamado_valido") &
            F.col("flag_valor_valido"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")
df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

df_silver = df_validos.select(
    "cd_custo",
    "cd_chamado", 
    "vl_custo",
    "dt_ingestion",
    "dc_origem"
)

In [0]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_custo = origem.cd_custo"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} atualizada com sucesso!")

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_invalidos)

print(f"✅ Tabela {nome_tabela_invalidos} atualizada para auditoria!")